In [ ]:
import numpy as np

### Load Dataset

In [ ]:
all_data = np.load('train_dev_test.npz')

In [ ]:
train_encoder_output = all_data['train_encoder_output']
train_decoder_input = all_data['train_decoder_input']
train_decoder_target = all_data['train_decoder_target'][:,1:,:]
validation_encoder_output = all_data['validation_encoder_output']
validation_decoder_input = all_data['validation_decoder_input']
validation_decoder_target = all_data['validation_decoder_target'][:,1:,:]
test_encoder_output = all_data['test_encoder_output']
test_decoder_input = all_data['test_decoder_input']
test_decoder_target = all_data['test_decoder_target'][:,1:,:]

In [ ]:
print("Train Encoder Output", train_encoder_output.shape)
print("Train Decoder Input", train_decoder_input.shape)
print("Train Decoder Target", train_decoder_target.shape)

In [ ]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)    
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

## Model Architecture

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, RepeatVector, Concatenate, Merge, Masking
from keras.layers import LSTM, GRU, Embedding, TimeDistributed, Bidirectional
from keras import backend as K
from keras import optimizers
from keras.utils import plot_model

#### Parameters

In [ ]:
emb_size = 300
lstm_size = 300
vocab_size = len(vocab)
max_length = train_decoder_target.shape[1]
lstm_layers = 2

lr = 0.001
dropout_rate = 0.2
batch_size = 64
n_epochs = 20

### Model

In [ ]:
K.clear_session()

In [ ]:
# Image -> Image embedding
image_input = Input(shape=(train_encoder_output.shape[1], ), name='image_input')
print("Image Input shape", image_input.shape)
img_emb = Dense(emb_size, activation='relu', name='img_embedding')(image_input)
print(img_emb.shape)

In [ ]:
# Sentence to Word embedding
caption_inputs = Input(shape=(None, ), name='caption_input')
print("Caption Input Shape", caption_inputs.shape)
emb_layer = Embedding(input_dim=vocab_size, output_dim=emb_size, name='Embedding')
word_emb = emb_layer(caption_inputs)
print(word_emb.shape)

In [ ]:
decoder_cell = LSTM(lstm_size, return_sequences=True, return_state=True, name='decoder')
encoder_states = [img_emb, img_emb]
decoder_out, _, _ = decoder_cell(word_emb, initial_state = encoder_states)

In [ ]:
output_layer = TimeDistributed(Dense(vocab_size, activation='softmax'))
decoder_out = output_layer(decoder_out)
print(decoder_out.shape)

In [ ]:
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1e-6)

In [ ]:
model = Model(inputs=[image_input,caption_inputs], outputs=[decoder_out])

model.compile(optimizer=rmsprop,
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
model.summary()
plot_model(model, to_file='model2.png', show_shapes=True)

In [ ]:
model.fit([train_encoder_output[:10,:], train_decoder_input[:10,:]], [train_decoder_target[:10,:,:]], 
#           validation_data=([validation_encoder_output, validation_decoder_input], [validation_decoder_target]),
           epochs=n_epochs, batch_size=batch_size, verbose=1)

## Inference

In [ ]:
encoder_model = Model(image_input, encoder_states)

In [ ]:
encoder_model.summary()

In [ ]:
decoder_state_input_h = Input(shape=(lstm_size, ))
decoder_state_input_c = Input(shape=(lstm_size, ))

In [ ]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_cell(word_emb, initial_state=decoder_states_inputs)

In [ ]:
decoder_states = [state_h, state_c]
decoder_outputs = output_layer(decoder_outputs)
decoder_model = Model([caption_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
decoder_model.summary()

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
states_value = encoder_model.predict(train_encoder_output[1,:,:])